In [1]:
# Import the dependencies.
import pandas as pd
import numpy as np
import time

In [2]:
import os
import sys

# Method to add the previous dir to the path found at:
# https://stackoverflow.com/questions/34478398/import-local-function-from-a-module-housed-in-another-directory-with-relative-im/35273613#35273613

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

# Import the requests library.
import requests

# Import the API key.
from config import weather_api_key

In [3]:
# Starting URL for Weather Map API Call.
url = "http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=" + weather_api_key
print(url)

http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=ed5c4a5d13114591e5ee76b3bbaf9589


In [4]:
# Create a set of random latitude and longitude combinations. zip packs the data in pairs
lats = np.random.uniform(low=-90.000, high=90.000, size=2000)
lngs = np.random.uniform(low=-180.000, high=180.000, size=2000)
lat_lngs = zip(lats, lngs)
lat_lngs

In [5]:
# Add the latitudes and longitudes to a list. This command is unpacking the zip array into a tuple list, 
# you can only do this once as it then removes it from memory
coordinates = list(lat_lngs)

In [6]:
# Use the citipy module to determine city based on latitude and longitude.
from citipy import citipy
# Create a list for holding the cities.
cities = []
# Identify the nearest city for each latitude and longitude combination.
for coordinate in coordinates:
    city = citipy.nearest_city(coordinate[0], coordinate[1]).city_name

    # If the city is unique, then we will add it to the cities list.
    if city not in cities:
        cities.append(city)
# Print the city count to confirm sufficient count.
len(cities)

757

In [7]:
# Create an empty list to hold the weather data.
city_data = []
# Print the beginning of the logging.
print("Beginning Data Retrieval     ")
print("-----------------------------")

# Create counters.
record_count = 1
set_count = 1

# Loop through all the cities in the list.
for i, city in enumerate(cities):

    # Group cities in sets of 50 for logging purposes.
    if (i % 50 == 0 and i >= 50):
        set_count += 1
        record_count = 1
        time.sleep(60)

    # Create endpoint URL with each city. city.replace(" ","+") takes out the spaces and concatonates the name together
    city_url = url + "&q=" + city.replace(" ","+")

    # Log the URL, record, and set numbers and the city.
    print(f"Processing Record {record_count} of Set {set_count} | {city}")
    # Add 1 to the record count.
    record_count += 1
    
# Run an API request for each of the cities.
    try:
        # Parse the JSON and retrieve data.
        city_weather = requests.get(city_url).json()
        # Parse out the needed data.
        city_lat = city_weather["coord"]["lat"]
        city_lng = city_weather["coord"]["lon"]
        city_max_temp = city_weather["main"]["temp_max"]
        city_humidity = city_weather["main"]["humidity"]
        city_description = city_weather["weather"][0]["description"]
        city_clouds = city_weather["clouds"]["all"]
        city_wind = city_weather["wind"]["speed"]
        city_country = city_weather["sys"]["country"]
        city_data.append({"City": city.title(),
                          "Lat": city_lat,
                          "Lng": city_lng,
                          "Max Temp": city_max_temp,
                          "Humidity": city_humidity,
                          "Cloudiness": city_clouds,
                          "Wind Speed": city_wind,
                          "Country": city_country,
                          "Current Description": city_description})

# If an error is experienced, skip the city.
    except:
        print("City not found. Skipping...")
        pass

# Indicate that Data Loading is complete.
print("-----------------------------")
print("Data Retrieval Complete      ")
print("-----------------------------")

Beginning Data Retrieval     
-----------------------------
Processing Record 1 of Set 1 | jimenez
Processing Record 2 of Set 1 | mataura
Processing Record 3 of Set 1 | vaini
Processing Record 4 of Set 1 | xinzhi
Processing Record 5 of Set 1 | avarua
Processing Record 6 of Set 1 | waipawa
Processing Record 7 of Set 1 | luau
Processing Record 8 of Set 1 | newala
Processing Record 9 of Set 1 | port alfred
Processing Record 10 of Set 1 | broome
Processing Record 11 of Set 1 | hobart
Processing Record 12 of Set 1 | jamestown
Processing Record 13 of Set 1 | tasiilaq
Processing Record 14 of Set 1 | dahod
Processing Record 15 of Set 1 | buraydah
Processing Record 16 of Set 1 | ribeira grande
Processing Record 17 of Set 1 | rikitea
Processing Record 18 of Set 1 | ponta delgada
Processing Record 19 of Set 1 | ushuaia
Processing Record 20 of Set 1 | new norfolk
Processing Record 21 of Set 1 | ballina
Processing Record 22 of Set 1 | butaritari
Processing Record 23 of Set 1 | taolanaro
City not fo

Processing Record 39 of Set 4 | saint-pierre
Processing Record 40 of Set 4 | lagoa
Processing Record 41 of Set 4 | georgetown
Processing Record 42 of Set 4 | changtu
Processing Record 43 of Set 4 | ozark
Processing Record 44 of Set 4 | half moon bay
Processing Record 45 of Set 4 | zhezkazgan
Processing Record 46 of Set 4 | port-cartier
Processing Record 47 of Set 4 | narsaq
Processing Record 48 of Set 4 | pisco
Processing Record 49 of Set 4 | namibe
Processing Record 50 of Set 4 | kirakira
Processing Record 1 of Set 5 | thurso
Processing Record 2 of Set 5 | laguna
Processing Record 3 of Set 5 | vila franca do campo
Processing Record 4 of Set 5 | miandrivazo
Processing Record 5 of Set 5 | forbesganj
Processing Record 6 of Set 5 | erzin
Processing Record 7 of Set 5 | mahebourg
Processing Record 8 of Set 5 | carballo
Processing Record 9 of Set 5 | vostok
Processing Record 10 of Set 5 | shatoy
Processing Record 11 of Set 5 | biak
Processing Record 12 of Set 5 | geraldton
Processing Record 

Processing Record 30 of Set 8 | olinda
Processing Record 31 of Set 8 | mount gambier
Processing Record 32 of Set 8 | quatre cocos
Processing Record 33 of Set 8 | dingle
Processing Record 34 of Set 8 | vila velha
Processing Record 35 of Set 8 | lixourion
Processing Record 36 of Set 8 | sabha
Processing Record 37 of Set 8 | volosovo
Processing Record 38 of Set 8 | san andres del rabanedo
Processing Record 39 of Set 8 | tabas
Processing Record 40 of Set 8 | te anau
Processing Record 41 of Set 8 | bilibino
Processing Record 42 of Set 8 | beringovskiy
Processing Record 43 of Set 8 | quartucciu
Processing Record 44 of Set 8 | west bay
Processing Record 45 of Set 8 | itarema
Processing Record 46 of Set 8 | zhangye
Processing Record 47 of Set 8 | nizhneyansk
City not found. Skipping...
Processing Record 48 of Set 8 | si bun ruang
Processing Record 49 of Set 8 | luena
Processing Record 50 of Set 8 | stornoway
Processing Record 1 of Set 9 | klaksvik
Processing Record 2 of Set 9 | sergeyevka
Proc

Processing Record 17 of Set 12 | saint andrews
Processing Record 18 of Set 12 | bur gabo
City not found. Skipping...
Processing Record 19 of Set 12 | ullapool
Processing Record 20 of Set 12 | eyl
Processing Record 21 of Set 12 | sao felix do xingu
Processing Record 22 of Set 12 | cheuskiny
City not found. Skipping...
Processing Record 23 of Set 12 | calama
Processing Record 24 of Set 12 | chom bung
City not found. Skipping...
Processing Record 25 of Set 12 | uttaradit
Processing Record 26 of Set 12 | tuy hoa
Processing Record 27 of Set 12 | midland
Processing Record 28 of Set 12 | tulum
Processing Record 29 of Set 12 | waitati
Processing Record 30 of Set 12 | turayf
Processing Record 31 of Set 12 | sobolevo
Processing Record 32 of Set 12 | uberaba
Processing Record 33 of Set 12 | koumac
Processing Record 34 of Set 12 | meridian
Processing Record 35 of Set 12 | liusha
Processing Record 36 of Set 12 | kieta
Processing Record 37 of Set 12 | agfalva
Processing Record 38 of Set 12 | lazaro 

Processing Record 7 of Set 16 | le port
-----------------------------
Data Retrieval Complete      
-----------------------------


In [8]:
# Convert the array of dictionaries to a Pandas DataFrame.
city_data_df = pd.DataFrame(city_data)
city_data_df.head(10)

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Current Description
0,Jimenez,28.3333,-105.4000,66.49,18,79,5.82,MX,broken clouds
1,Mataura,-46.1927,168.8643,43.68,98,81,3.22,NZ,broken clouds
2,Vaini,-21.2000,-175.2000,75.36,100,40,0.00,TO,scattered clouds
3,Xinzhi,36.4989,111.7047,72.52,23,0,4.94,CN,clear sky
4,Avarua,-21.2078,-159.7750,82.45,78,40,10.36,CK,scattered clouds
5,Waipawa,-41.4122,175.5153,45.64,83,0,4.32,NZ,clear sky
6,Luau,-10.7073,22.2247,70.00,90,88,2.33,AO,overcast clouds
7,Newala,-10.9431,39.2787,69.76,95,91,5.59,TZ,moderate rain
8,Port Alfred,-33.5906,26.8910,65.17,85,99,4.00,ZA,overcast clouds
9,Broome,42.2506,-75.8330,45.19,77,100,8.05,US,overcast clouds


In [9]:
new_column_order = ["City", "Country", "Lat", "Lng", "Max Temp","Humidity","Cloudiness","Wind Speed","Current Description"]
city_data_df=city_data_df[new_column_order]
city_data_df.head(10)

,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,Jimenez,MX,28.3333,-105.4000,66.49,18,79,5.82,broken clouds
1,Mataura,NZ,-46.1927,168.8643,43.68,98,81,3.22,broken clouds
2,Vaini,TO,-21.2000,-175.2000,75.36,100,40,0.00,scattered clouds
3,Xinzhi,CN,36.4989,111.7047,72.52,23,0,4.94,clear sky
4,Avarua,CK,-21.2078,-159.7750,82.45,78,40,10.36,scattered clouds
5,Waipawa,NZ,-41.4122,175.5153,45.64,83,0,4.32,clear sky
6,Luau,AO,-10.7073,22.2247,70.00,90,88,2.33,overcast clouds
7,Newala,TZ,-10.9431,39.2787,69.76,95,91,5.59,moderate rain
8,Port Alfred,ZA,-33.5906,26.8910,65.17,85,99,4.00,overcast clouds
9,Broome,US,42.2506,-75.8330,45.19,77,100,8.05,overcast clouds


In [10]:
# Create the output file (CSV).
output_data_file = "WeatherPy_database.csv"
# Export the City_Data into a CSV.
city_data_df.to_csv(output_data_file, index_label="City_ID")